In [1]:
#configurando o ambiente 
!apt-get update -qq
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark


In [2]:
#criando as pastas do Java e Spark
import os
os.environ['JAVA_HOME']="/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME']='/content/spark-3.1.2-bin-hadoop2.7'

In [ ]:
!pip install -q findspark

In [3]:
#Esse pacote referência o spark
import findspark

In [4]:
findspark.init()

In [5]:
#importa o spark sessão
from pyspark.sql import SparkSession

In [6]:
#Configuração da sessão
spark= SparkSession.builder \
    .master('local[*]') \
    .appName('Aula_spark') \
    .config('spark.iu.port','4050') \
    .getOrCreate()

In [ ]:
#!wget -q https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
#!unzip ngrok-stable-linux-amd64.zip

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [ ]:

#get_ipython().system_raw('ngrok config add-authtoken 2PieZgGtQEXUx4JnerL86qjMTaI_23TviCWZVtdBFfJ4NGGj')
#get_ipython().system_raw('./ngrok http 4050 &')*/

In [ ]:
#!curl -s http://localhost:4050/api/tunnels

In [7]:
#criando uma lista de tupla
data=[('Diego',26),('Josi',23)]

In [8]:
#criando uma lista de colunas
Col=['Nome','Idade']

In [9]:
#Criando um data frame no spark
df=spark.createDataFrame(data,Col)

In [10]:
#Mostrando o data frame 
df.show()

+-----+-----+
| Nome|Idade|
+-----+-----+
|Diego|   26|
| Josi|   23|
+-----+-----+



In [11]:
#Monstrando o data frame e modelo de Data frame pandas(py)
df.toPandas()

,Nome,Idade
0,Diego,26
1,Josi,23


In [12]:
#criando uma conexão com o Drive caso queira se não desconsidere esse passo 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
#biblioteca de descompactação
import zipfile

In [16]:
#extraindo o arquivo do zip para a mesma pasta 
zipfile.ZipFile('/content/drive/MyDrive/MBA_03/empresas.zip','r').extractall('/content/drive/MyDrive/MBA_03')

In [18]:
#criando um caminho
path='/content/drive/MyDrive/MBA_03/empresas'

In [20]:
#lendo csv 
empresas=spark.read.csv(path,sep=';',inferSchema=True)

In [42]:
#mostrar as 20 primeiras linhas, dentro do parentes de show podemos escolher quantas linhas vamos
#mostrar igual a função head do python
empresas.show()

+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|        razao_social|Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|        306|FRANCAMAR REFRIGE...|             2240|                      49|          0,00|            1|                       null|
|       1355|BRASILEIRO & OLIV...|             2062|                      49|          0,00|            5|                       null|
|       4820|REGISTRO DE IMOVE...|             3034|                      32|          0,00|            5|                       null|
|       5347|ROSELY APARECIDA ...|             2135|                      50|          0,00|            5|                       null|
|       6846|BADU E FILHOS TEC...|             2062|   

In [25]:
#mostrando a quantidade de linhas no Data frame
empresas.count()

4585679

In [31]:
#criando uma lista com nome da colunas
Names=['cnpj_basico','razao_social','Natureza_juridica','qualificacao_responsavel','capital_social','porte_empresa','ente_federativo_responsável']

In [40]:
#renomeando as colunas
for index, ColNames in enumerate(Names):
  empresas=empresas.withColumnRenamed(f"_c{index}",ColNames)


In [41]:
#Colunas após a transformação
empresas.columns

['cnpj_basico',
 'razao_social',
 'Natureza_juridica',
 'qualificacao_responsavel',
 'capital_social',
 'porte_empresa',
 'ente_federativo_responsável']

In [45]:
#Vendo o tipo de dados dentro do data frame
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- Natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: string (nullable = true)
 |-- porte_empresa: integer (nullable = true)
 |-- ente_federativo_responsável: string (nullable = true)



In [51]:
#importando double,string e Datatype
from pyspark.sql.types import DoubleType, StringType,DateType

In [52]:
# importando funções matemáticas
from pyspark.sql import functions as f

In [57]:
#Mudando ',' para '.'
empresas=empresas.withColumn('capital_social',f.regexp_replace('capital_social',',','.'))

In [60]:
empresas.show( )



+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|        razao_social|Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|        306|FRANCAMAR REFRIGE...|             2240|                      49|          0.00|            1|                       null|
|       1355|BRASILEIRO & OLIV...|             2062|                      49|          0.00|            5|                       null|
|       4820|REGISTRO DE IMOVE...|             3034|                      32|          0.00|            5|                       null|
|       5347|ROSELY APARECIDA ...|             2135|                      50|          0.00|            5|                       null|
|       6846|BADU E FILHOS TEC...|             2062|   

In [66]:
#Transformando string para Double
empresas=empresas.withColumn('capital_social',empresas['capital_social'].cast(DoubleType()))

In [67]:
empresas.printSchema()

root
 |-- cnpj_basico: integer (nullable = true)
 |-- razao_social: string (nullable = true)
 |-- Natureza_juridica: integer (nullable = true)
 |-- qualificacao_responsavel: integer (nullable = true)
 |-- capital_social: double (nullable = true)
 |-- porte_empresa: integer (nullable = true)
 |-- ente_federativo_responsável: string (nullable = true)



In [79]:
empresas \
  .select('*') \
  .show(5,False)

+-----------+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|razao_social                                                                                |Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+--------------------------------------------------------------------------------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|306        |FRANCAMAR REFRIGERACAO TECNICA S/C LTDA                                                     |2240             |49                      |0.0           |1            |null                       |
|1355       |BRASILEIRO & OLIVEIRA LTDA                                                                  |2062             |49                      |0.0           |5       

In [81]:
#usando filtro where

empresas \
  .where('capital_social>20 and capital_social<100') \
  .count()

8729

In [75]:
# Usando filtro 'filter'
empresas \
  .filter('capital_social>20 and capital_social<100') \
  .show()

+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|cnpj_basico|        razao_social|Natureza_juridica|qualificacao_responsavel|capital_social|porte_empresa|ente_federativo_responsável|
+-----------+--------------------+-----------------+------------------------+--------------+-------------+---------------------------+
|   17350147|ERIK MARCELO DOS ...|             2135|                      50|          50.0|            1|                       null|
|   17833214|ALEXANDRE MACHADO...|             2135|                      50|          50.0|            1|                       null|
|   20860830|YASMIN MOURA DA F...|             2135|                      50|          50.0|            1|                       null|
|   22242856|JOAO CESAR MESSIA...|             2135|                      50|          50.0|            1|                       null|
|   22510243|DAIANE DE JESUS S...|             2135|   

In [93]:
#usando comando SQL no spark
empresas \
  .select('razao_social','porte_empresa') \
  .where('capital_social>20 and capital_social<100') \
  .show(5)

+--------------------+-------------+
|        razao_social|porte_empresa|
+--------------------+-------------+
|ERIK MARCELO DOS ...|            1|
|ALEXANDRE MACHADO...|            1|
|YASMIN MOURA DA F...|            1|
|JOAO CESAR MESSIA...|            1|
|DAIANE DE JESUS S...|            1|
+--------------------+-------------+
only showing top 5 rows



In [99]:
#orderBy
empresas \
  .select('razao_social','porte_empresa') \
  .where('capital_social>20 and capital_social<100') \
  .orderBy('razao_social',ascending =True) \
  .show(5,True)


+--------------------+-------------+
|        razao_social|porte_empresa|
+--------------------+-------------+
|3 MOSQUETEIROS PU...|            5|
|A P NERES & CIA LTDA|            1|
|A. C. PIPA ARTESA...|            1|
|ABEL ALVES DIAS 5...|            1|
|ABEL GONCALVES DO...|            1|
+--------------------+-------------+
only showing top 5 rows



In [104]:
#sumarizando os dados com a função agg
empresas \
    .select('*')\
    .groupBy('porte_empresa')\
    .agg(
        f.avg('capital_social')
    ) \
    .orderBy('porte_empresa', ascending=True)\
    .show()

+-------------+-------------------+
|porte_empresa|avg(capital_social)|
+-------------+-------------------+
|         null|   8.35421888053467|
|            1| 339994.53313506936|
|            3| 2601001.7677092673|
|            5|  708660.4208249798|
+-------------+-------------------+



In [106]:
#quantas empresas não tem capital social
empresas \
    .select('*')\
    .filter('capital_social==0') \
    .groupBy('porte_empresa')\
    .agg(
        f.count('capital_social')
    ) \
    .orderBy('porte_empresa', ascending=True)\
    .show()

+-------------+---------------------+
|porte_empresa|count(capital_social)|
+-------------+---------------------+
|         null|                 5984|
|            1|               457984|
|            3|                12257|
|            5|              1214798|
+-------------+---------------------+

